In [2]:
!pip install -qU langchain_community
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 89.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.3 MB/s eta 0:00:00:00:01


In [3]:
import re
from langchain_community.document_loaders import PDFPlumberLoader

In [4]:
pdf_path = "/kaggle/input/test-res-regex/Steve Sun - Resume (2).pdf"

In [26]:
def extract_text_pdf(pdf_path):
        loader = PDFPlumberLoader(pdf_path)
        docs = loader.load()
        text = ''
        for doc in docs:
            text += doc.page_content # Append extracted text
        return text 


def extract_name_from_resume(text):
    name = None

    # Use regex pattern to find a potential name
    pattern = r"(\b[A-Z][a-z]+\b)\s(\b[A-Z][a-z]+\b)"
    match = re.search(pattern, text)
    if match:
        name = match.group()

    return name

def extract_email_from_resume(text):
    email = None

    # Use regex pattern to find a potential email address
    pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    match = re.search(pattern, text)
    if match:
        email = match.group()

    return email


def extract_skills_from_resume(text, skills_list):
    skills = []

    # Search for skills in the resume text
    for skill in skills_list:
        pattern = r"\b{}\b".format(re.escape(skill))
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            skills.append(skill)

    return skills


def extract_education_details(text):

    pattern = r"(?P<university>[\w\s,&]+(?:University|College|Institute|Academy))\n(?P<degree>.+?)\s(?:-\s(?P<gpa>\d\.\d{1,2}\sGPA))?\s(?P<start_date>\w+\s\d{4})\s(?:[-–]\s(?P<end_date>\w+\s\d{4}))?"

    matches = re.finditer(pattern, text)
    
    education_dict = {
        "University": [],
        "Degree": [],
        "GPA": [],
        "Start Date": [],
        "End Date": [],
    }
    
    # Extract and organize data
    for match in matches:
        education_dict["University"].append(match.group("university").strip())
        education_dict["Degree"].append(match.group("degree").strip())
        education_dict["GPA"].append(match.group("gpa").strip() if match.group("gpa") else "N/A")
        education_dict["Start Date"].append(match.group("start_date").strip())
        education_dict["End Date"].append(match.group("end_date").strip() if match.group("end_date") else "Present")
    
    return education_dict


def extracting_experirnce(text):
        
    matches = re.search(pattern, text, re.DOTALL)
    
    if matches:
        work = matches.group(1).strip()
        return work


In [34]:
text = extract_text_pdf(pdf_path)
skills_list = ['Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Project Management', 'Deep Learning', 'SQL', 'Tableau','AI',
'Data Visualization','Statistical Analysis','Big Data','Cloud Technologies''Problem-Solving','Critical Thinking',
'Storytelling']
pattern = r"WORK EXPERIENCE\n(.*?)\nEDUCATION"

# below is the general pattern for the experience extraction
# pattern =  r"(?i)(work\s*experience|professional\s*experience|employment\s*history)[\s\S]*?(?=(education|skills|projects|$))"

In [29]:
output = {
    "Name": extract_name_from_resume(text),
    "Email": extract_email_from_resume(text),
    "Skills":extract_skills_from_resume(text, skills_list),
    "Education": extract_education_details(text),
    "Experience": extracting_experirnce(text),
}

In [33]:
print(output)

{'Name': 'Steve Sun', 'Email': 'stevesun1245@gmail.com', 'Skills': ['Python', 'Data Analysis', 'Machine Learning', 'Communication', 'Deep Learning', 'SQL', 'AI', 'Data Visualization', 'Statistical Analysis', 'Big Data', 'Critical Thinking', 'Storytelling'], 'Education': {'University': ['EDUCATION\nSkyline University'], 'Degree': ['Master of Science in Data Science'], 'GPA': ['3.81 GPA'], 'Start Date': ['August 2018'], 'End Date': ['May 2020']}, 'Experience': {'work': ['DataNova Insights\nData Scientist March 2021 – Present\n• Developed and deployed machine learning models to optimize customer retention, increasing\nengagement by 15%.\n• Conducted exploratory data analysis (EDA) and statistical testing to derive actionable business\ninsights.\n• Built scalable ETL pipelines to process large datasets using Python and SQL.\n• Collaborated with cross-functional teams to improve decision-making through predictive analytics.\nNeural Sphere Labs\nData Scientist June 2019 – December 2023\n• De